In [1]:
# load data
import pandas as pd
aapl = pd.read_csv("AAPL_market_data.csv", sep=",", header=0)
msft = pd.read_csv("MSFT_market_data.csv", sep=",", header=0)
tsla = pd.read_csv("TSLA_market_data.csv", sep=",", header=0)
stock = pd.read_csv("Stock_Sectors.csv", sep=",", header=0)

FileNotFoundError: [Errno 2] No such file or directory: 'AAPL_market_data.csv'

In [ ]:
# combine dataframes
market_combined_df= pd.concat(
    [aapl, msft, tsla], 
    keys=["AAPL" , "MSFT", "TSLA"], 
    names = ["Symbol"])
print(market_combined_df.head())
print(market_combined_df.info())

                  Date,Open,High,Low,Close,Adj Close,Volume
Symbol                                                     
AAPL   0  2023-01-02,150.12374103444336,154.672945308214...
       1  2023-01-03,149.85872969261115,154.399903319659...
       2  2023-01-04,151.94982405342844,156.554364176259...
       3  2023-01-05,156.73025625166827,161.479657956264...
       4  2023-01-06,156.15300813946828,160.884917477027...
<class 'pandas.DataFrame'>
MultiIndex: 180 entries, ('AAPL', 0) to ('TSLA', 59)
Data columns (total 1 columns):
 #   Column                                     Non-Null Count  Dtype
---  ------                                     --------------  -----
 0   Date,Open,High,Low,Close,Adj Close,Volume  180 non-null    str  
dtypes: str(1)
memory usage: 2.0+ KB
None


In [ ]:
# load the wrong column
market_series = market_combined_df.iloc[:, 0]

# split the volume column by commas
df_fixed = market_series.str.split(",", expand=True)

#rename the columns
df_fixed.columns = ["Date","Open","High","Low","Close","Adj Close","Volume"]

# convert the date column into datetime format
df_fixed["Date"] = pd.to_datetime(df_fixed["Date"])

# convert the other columns into numeric format (float type)
for col in df_fixed.columns[1:]:
    df_fixed[col] = df_fixed[col].astype(float)

df_fixed.head()

Date        Open        High         Low       Close  \
Symbol                                                                
AAPL   0 2023-01-02  150.123741  154.672945  148.607340  151.640142   
       1 2023-01-03  149.858730  154.399903  148.345005  151.372454   
       2 2023-01-04  151.949824  156.554364  150.414977  153.484671   
       3 2023-01-05  156.730256  161.479658  155.147122  158.313390   
       4 2023-01-06  156.153008  160.884917  154.575705  157.730311   

           Adj Close     Volume  
Symbol                           
AAPL   0  148.607340  4521441.0  
       1  148.345005  3935840.0  
       2  150.414977  1351279.0  
       3  155.147122  4011062.0  
       4  154.575705  2870230.0

In [ ]:
# check the column names
df_fixed.info()

<class 'pandas.DataFrame'>
MultiIndex: 180 entries, ('AAPL', 0) to ('TSLA', 59)
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       180 non-null    datetime64[us]
 1   Open       180 non-null    float64       
 2   High       180 non-null    float64       
 3   Low        180 non-null    float64       
 4   Close      180 non-null    float64       
 5   Adj Close  180 non-null    float64       
 6   Volume     180 non-null    float64       
dtypes: datetime64[us](1), float64(6)
memory usage: 10.4+ KB


In [ ]:
# reset the index to get the symbol column back
df_fixed["Symbol"] = market_combined_df.index.get_level_values(0)

<class 'pandas.DataFrame'>
DatetimeIndex: 180 entries, 2023-01-02 to 2023-03-24
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       180 non-null    float64
 1   High       180 non-null    float64
 2   Low        180 non-null    float64
 3   Close      180 non-null    float64
 4   Adj Close  180 non-null    float64
 5   Volume     180 non-null    float64
 6   Symbol     180 non-null    str    
dtypes: float64(6), str(1)
memory usage: 11.2 KB


In [ ]:
# check the reset index
df_fixed.reset_index(inplace=True)
df_fixed.info()

<class 'pandas.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   index      180 non-null    int64         
 1   Date       180 non-null    datetime64[us]
 2   Open       180 non-null    float64       
 3   High       180 non-null    float64       
 4   Low        180 non-null    float64       
 5   Close      180 non-null    float64       
 6   Adj Close  180 non-null    float64       
 7   Volume     180 non-null    float64       
 8   Symbol     180 non-null    str           
dtypes: datetime64[us](1), float64(6), int64(1), str(1)
memory usage: 12.8 KB


In [ ]:
# drop the old index column
df_fixed.drop(columns = ["index"] , inplace = True)

In [ ]:
# set the new multi index
df_fixed.set_index(["Symbol","Date"] , inplace = True)

In [ ]:
# check the final result
df_fixed.head()
df_fixed.info()

<class 'pandas.DataFrame'>
MultiIndex: 180 entries, ('AAPL', Timestamp('2023-01-02 00:00:00')) to ('TSLA', Timestamp('2023-03-24 00:00:00'))
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       180 non-null    float64
 1   High       180 non-null    float64
 2   Low        180 non-null    float64
 3   Close      180 non-null    float64
 4   Adj Close  180 non-null    float64
 5   Volume     180 non-null    float64
dtypes: float64(6)
memory usage: 9.4+ KB


In [ ]:
# resample the data to weekly frequency and calculate the mean of the volume and the last closing price for each week
weekly = (
    df_fixed
    .groupby(level="Symbol")
    .resample("W", level="Date")
    .agg({
        "Volume": "mean",
        "Close": "last"
    })
)

In [123]:
# check the weekly data for AAPL
weekly.loc["AAPL"]

,Volume,Close
Date,,
2023-01-08,3337970.4,157.730311
2023-01-15,3736733.8,165.477451
2023-01-22,2845015.0,152.315667
2023-01-29,1999904.2,142.385342
2023-02-05,2788281.2,141.091536
2023-02-12,3390676.0,137.402516
2023-02-19,3099890.2,140.728006
2023-02-26,3356615.6,130.110558
2023-03-05,3350931.0,128.138709


In [78]:
# resample the data to monthly frequency and calculate the mean of the volume and the last closing price for each month
Monthly = (
    df_fixed
    .groupby(level="Symbol")
    .resample("ME", level="Date")
    .agg({
        "Volume": "mean",
        "Close": "last"
    })
)

In [124]:
# check the monthly data for AAPL
Monthly.loc["AAPL"]

,Volume,Close
Date,,
2023-01-31,2.952221e+06,146.185732
2023-02-28,3.320702e+06,132.747446
2023-03-31,3.046214e+06,130.991922


In [ ]:
# calculate the 20 day simple moving average of the adjusted closing price for each stock
df_fixed["SMA_20"] = (df_fixed.groupby(level = "Symbol")["Adj Close"]
                      .rolling(window =20)
                      .mean()
                      .reset_index(level = 0, drop = True)
)

In [83]:
# check the result
df_fixed.loc["AAPL"][["Adj Close", "SMA_20"]].head(25)

,Adj Close,SMA_20
Date,,
2023-01-02,148.607340,NaN
2023-01-03,148.345005,NaN
2023-01-04,150.414977,NaN
2023-01-05,155.147122,NaN
2023-01-06,154.575705,NaN
2023-01-09,154.006443,NaN
2023-01-10,159.024628,NaN
2023-01-11,161.624474,NaN
2023-01-12,160.268527,NaN


In [ ]:
# calculate the maximam high price and the average volume for each stock
kpis = df_fixed.groupby(level = "Symbol").agg({
    "High": "max",
    "Volume": "mean"
})

In [86]:
# rename column names 
kpis.rename(columns = {
    "High": "Max High Price",
    "Volume": "Average Volume"
}, inplace = True)

In [114]:
#filter the stock with 
filltered_kpis = kpis[kpis["Average Volume"] >= 500_000]

In [ ]:
# check the result
kpis.head()

,Max High Price,Average Volume
Symbol,,
AAPL,168.787000,3103246.20
MSFT,335.332350,3081944.85
TSLA,241.439292,3081944.85


In [ ]:
# check the result
kpis["Max High Price"].equals(
    df_fixed.groupby("Symbol")["High"].max()
)

True

In [125]:
# reset the index to prepare for pivot table
df_fixed = df_fixed.reset_index()

In [126]:
# check the column names
df_fixed.info()

<class 'pandas.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   index           180 non-null    int64         
 1   Symbol          180 non-null    category      
 2   Date            180 non-null    datetime64[us]
 3   Open            180 non-null    float64       
 4   High            180 non-null    float64       
 5   Low             180 non-null    float64       
 6   Close           180 non-null    float64       
 7   Adj Close       180 non-null    float64       
 8   Volume          180 non-null    float64       
 9   SMA_20          123 non-null    float64       
 10  Intraday Range  180 non-null    float64       
dtypes: category(1), datetime64[us](1), float64(8), int64(1)
memory usage: 14.5 KB


In [108]:
# the pivot table to show the adjusted closing price for each stock on each date
pivot_table = pd.pivot_table(
    df_fixed,
    values="Adj Close",
    index="Date",
    columns="Symbol",
    margins=True
)
pivot_table.head()

Symbol,AAPL,MSFT,TSLA,All
Date,,,,
2023-01-02 00:00:00,148.607340,251.758140,181.265861,193.877113
2023-01-03 00:00:00,148.345005,248.132654,178.655511,191.711057
2023-01-04 00:00:00,150.414977,246.811197,177.704062,191.643412
2023-01-05 00:00:00,155.147122,242.167146,174.360345,190.558205
2023-01-06 00:00:00,154.575705,237.222983,170.800548,187.533079


In [ ]:
# check the result
df_fixed.columns

Index(['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'SMA_20'],
      dtype='str')

In [101]:
# convert symbol column to category type to save memory
df_fixed["Symbol"] = df_fixed["Symbol"].astype("category")
df_fixed.info()

<class 'pandas.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Symbol     180 non-null    category      
 1   Date       180 non-null    datetime64[us]
 2   Open       180 non-null    float64       
 3   High       180 non-null    float64       
 4   Low        180 non-null    float64       
 5   Close      180 non-null    float64       
 6   Adj Close  180 non-null    float64       
 7   Volume     180 non-null    float64       
 8   SMA_20     123 non-null    float64       
dtypes: category(1), datetime64[us](1), float64(7)
memory usage: 11.6 KB


In [ ]:
# check the memory usage of dataframe 
df_fixed.memory_usage(deep=True)

Index         132
Symbol        339
Date         1440
Open         1440
High         1440
Low          1440
Close        1440
Adj Close    1440
Volume       1440
SMA_20       1440
dtype: int64

In [ ]:
#Vectorized Intraday Range
df_fixed["Intraday Range"] = df_fixed["High"] - df_fixed["Low"]
# check the result
df_fixed[["High", "Low", "Intraday Range"]].head()

,High,Low,Intraday Range
0,154.672945,148.607340,6.065606
1,154.399903,148.345005,6.054898
2,156.554364,150.414977,6.139387
3,161.479658,155.147122,6.332536
4,160.884917,154.575705,6.309212


In [ ]:
# check for missing values
df_fixed.isna().sum()

Symbol             0
Date               0
Open               0
High               0
Low                0
Close              0
Adj Close          0
Volume             0
SMA_20            57
Intraday Range     0
dtype: int64